# Домашнее задание №6

Урок 6. Рекуррентные нейронные сети. LSTM. GRU.

Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

## Загрузка данных и библиотек

In [1]:
!pip install stop_words
!pip install pymorphy2 

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=a6a454c3fcc60d8495bbf2afe6caad0f16e1b54639b66541f9011a174c96b02e
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 6.4MB/s 


In [13]:
from sklearn import model_selection, preprocessing, metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re
import pandas, xgboost, numpy, textblob, string
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import keras
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.objectives import categorical_crossentropy
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import EarlyStopping
from keras import optimizers
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Устанавливаем гиперпараметры
epochs = 10
batch_size = 512

## Предобработка данных

In [4]:
data = pd.read_excel('summer.xls')

In [5]:
exclude = set(punctuation)
stop_words = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in stop_words]
    return " ".join(txt)

# Делаем предобработку и сокращаем количество классов до 2
data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [7]:
text_corpus_train = X_train.values
text_corpus_test = X_test.values

## Токенизация и векторизация

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [9]:
y_train = y_train.values
y_test = y_test.values

## Обучение RNN

In [10]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 5s 190ms/step - loss: 0.4368 - auc: 0.6878 - val_loss: 0.3264 - val_auc: 0.9028
Epoch 2/10
28/28 [==============================] - 5s 176ms/step - loss: 0.2862 - auc: 0.8713 - val_loss: 0.2518 - val_auc: 0.9317
Epoch 3/10
28/28 [==============================] - 5s 184ms/step - loss: 0.2091 - auc: 0.9368 - val_loss: 0.2172 - val_auc: 0.9486
Epoch 4/10
28/28 [==============================] - 5s 175ms/step - loss: 0.1404 - auc: 0.9737 - val_loss: 0.2089 - val_auc: 0.9505
Epoch 5/10
28/28 [==============================] - 5s 179ms/step - loss: 0.0992 - auc: 0.9862 - val_loss: 0.2391 - val_auc: 0.9436


In [49]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 16ms/step - loss: 0.2037 - auc_9: 0.9539


Test score: 0.2037380039691925
Test accuracy: 0.9538590908050537


## Обучение LSTM

In [46]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 14s 518ms/step - loss: 0.4775 - auc_8: 0.6058 - val_loss: 0.3323 - val_auc_8: 0.9106
Epoch 2/10
28/28 [==============================] - 14s 496ms/step - loss: 0.2856 - auc_8: 0.8825 - val_loss: 0.2410 - val_auc_8: 0.9288
Epoch 3/10
28/28 [==============================] - 14s 501ms/step - loss: 0.2097 - auc_8: 0.9426 - val_loss: 0.2100 - val_auc_8: 0.9491
Epoch 4/10
28/28 [==============================] - 14s 491ms/step - loss: 0.1513 - auc_8: 0.9715 - val_loss: 0.2064 - val_auc_8: 0.9501
Epoch 5/10
28/28 [==============================] - 14s 503ms/step - loss: 0.1091 - auc_8: 0.9840 - val_loss: 0.2178 - val_auc_8: 0.9458


In [47]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 39ms/step - loss: 0.1973 - auc_8: 0.9545


Test score: 0.19731782376766205
Test accuracy: 0.9545498490333557


## Обучение GRU

In [50]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 2s 89ms/step - loss: 0.5164 - auc_10: 0.5702 - val_loss: 0.3270 - val_auc_10: 0.9105
Epoch 2/10
28/28 [==============================] - 1s 45ms/step - loss: 0.2831 - auc_10: 0.9039 - val_loss: 0.2386 - val_auc_10: 0.9302
Epoch 3/10
28/28 [==============================] - 1s 46ms/step - loss: 0.2025 - auc_10: 0.9463 - val_loss: 0.2136 - val_auc_10: 0.9452
Epoch 4/10
28/28 [==============================] - 1s 46ms/step - loss: 0.1479 - auc_10: 0.9715 - val_loss: 0.2137 - val_auc_10: 0.9471


In [51]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 21ms/step - loss: 0.1948 - auc_10: 0.9570


Test score: 0.19481970369815826
Test accuracy: 0.9570112228393555


Наилучший результат дает модель GRU, при этом она обучается быстрее LSTM. RNN дает высокий результат, потому что последовательности имеют небольшой размер